In [30]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import json
import os

In [31]:
xml_path = "/Users/nakamura/git/hori/lab1/tei/DKB01.xml"

In [ ]:
soup = BeautifulSoup(open(xml_path), "xml")

In [ ]:
entries = soup.find_all("div", type="diary-entry")

len(entries)

4259

In [ ]:
entity_xml_path = "/Users/nakamura/git/hori/lab1/tei/entity.xml"

entity_soup = BeautifulSoup(open(entity_xml_path), "xml")

In [ ]:
def get_structured_data(html_text):

    data = []

    # BeautifulSoupでHTMLをパース
    soup = BeautifulSoup(html_text, "html.parser")

    p_str = str(soup.find("p").decode_contents())

    segments = p_str.split("<time")

    for segment in segments:

        item = {}
        data.append(item)

        if "</time>" in segment:
            segment = "<time" + segment

        item["text"] = segment

        segment_soup = BeautifulSoup(segment, "html.parser")

        # print(segment_soup.get_text())

        time = segment_soup.find("time")

        

        if time is not None:
            # print(time.get("when"), segment_soup.get_text())
            item["headline"] = time.get("when")

        places = segment_soup.find_all("placename")

        for place in places:
            # print("*", place.get_text().strip())

            place_id = place.get("corresp")[1:]

            # print(place_id)

            entity = entity_soup.find("place", {"xml:id": place_id})

            if entity is None:
                continue

            geo = entity.find("geo")

            if geo is not None:
                lat, lng = geo.text.strip().split(" ")

                # print(place_id, lat, lng)

                item["lat"] = lat
                item["lng"] = lng

                break

    return data

In [ ]:
data_list = []

for entry in tqdm(entries): # [0:10]

    id = entry.get("xml:id")

    date = entry.find("date")

    if date is None:
        continue

    when = date.get("when")

    p = entry.find("p")

    text = str(p)

    

    data_list.append(when)

    

    data = get_structured_data(text)

    slides = []

    slides.append({
        "date": "",
        "text": {
            "headline": when,
            "text": ""
        },
        "location": {
            "line": True
        },
        "type": "overview"
    })

    for item in data:
            
        slide = {
            "date": "",
            "text": {
                "headline": item["headline"] if "headline" in item else "",
                "text": item["text"]
            },
            # "type": "overview"
        }

        if "lat" in item:
            slide["location"] = {
                "line": True,
                "lat": float(item["lat"]),
                "lon": float(item["lng"]),
                "zoom": 8
            }

        slides.append(slide)

    final_data = {
        "storymap": {
            "call_to_action": True,
            "call_to_action_text": "",
            "map_as_image": False,
            "slides": slides,
            "zoomify": False,
            "map_type": "osm:standard",
            "map_subdomains": "",
            "attribution": "",
            "language": "jp"
        },
         "font_css": "stock:opensans-gentiumbook"
    }

    output_path = f"../public/data/storymap/date/{when}.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    with open(output_path, "w") as f:

        f.write(json.dumps(final_data, ensure_ascii=False, indent=4))

opath = "../src/data/storymap/list.json"

os.makedirs(os.path.dirname(opath), exist_ok=True)

with open(opath, "w") as f:

    f.write(json.dumps(data_list, ensure_ascii=False, indent=4))

100%|██████████| 4259/4259 [05:32<00:00, 12.81it/s]
